In [ ]:
! unzip '/content/OneDrive_2022-07-27.zip'   #unzipping the file

# Importing the required library


In [2]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [4]:
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model 

In [5]:
import tensorflow as tf


# Data Preparation

In [6]:
def read_files(file):
    notes=[]
    notes_to_parse=None
    #parse the midi file
    midi=converter.parse(file)
    #seperate all instruments from the file
    instrmt=instrument.partitionByInstrument(midi)
    for part in instrmt.parts:
        #fetch data only of Piano instrument
        if 'Piano' in str(part):
            notes_to_parse=part.recurse()
            for element in notes_to_parse:
                if type(element)==note.Note:
                    notes.append(str(element.pitch))
                elif type(element)==chord.Chord:
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes
#retrieve paths recursively from inside the directories/files
file_path=['schubert'] 
all_files=glob.glob('/content/All Midi Files/'+file_path[0]+'/*.mid',recursive=True)
notes_array = np.array([read_files(i) for i in all_files])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [7]:
#unique notes
notess = sum(notes_array,[]) 
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(50,200,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

Unique Notes: 304

Frequency notes
50 : 167
70 : 146
90 : 137
110 : 128
130 : 121
150 : 117
170 : 109
190 : 102


In [8]:
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [9]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [10]:
x_train.shape

(51170, 50, 1)

# Model Buiding

In [11]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
import tensorflow as tf

In [13]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [14]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 167)               42919     
                                                                 
Total params: 898,215
Trainable params: 898,215
Non-trai

In [15]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
with tf.device('/gpu:0'):
    model.fit(
        x_train,y_train,
        batch_size=128,epochs=80, 
        validation_data=(x_test,y_test))

Epoch 1/80
400/400 [==============================] - 15s 20ms/step - loss: 4.6709 - accuracy: 0.0307 - val_loss: 4.5304 - val_accuracy: 0.0460
Epoch 2/80
400/400 [==============================] - 7s 18ms/step - loss: 4.4302 - accuracy: 0.0566 - val_loss: 4.3568 - val_accuracy: 0.0718
Epoch 3/80
400/400 [==============================] - 7s 18ms/step - loss: 4.1853 - accuracy: 0.0883 - val_loss: 4.0698 - val_accuracy: 0.1023
Epoch 4/80
400/400 [==============================] - 7s 18ms/step - loss: 3.9248 - accuracy: 0.1204 - val_loss: 3.8433 - val_accuracy: 0.1334
Epoch 5/80
400/400 [==============================] - 7s 18ms/step - loss: 3.6809 - accuracy: 0.1506 - val_loss: 3.6284 - val_accuracy: 0.1659
Epoch 6/80
400/400 [==============================] - 7s 18ms/step - loss: 3.4508 - accuracy: 0.1857 - val_loss: 3.4645 - val_accuracy: 0.1890
Epoch 7/80
400/400 [==============================] - 7s 18ms/step - loss: 3.2472 - accuracy: 0.2168 - val_loss: 3.3284 - val_accuracy: 0.212

In [16]:
model.save("MOD")

INFO:tensorflow:Assets written to: MOD/assets


INFO:tensorflow:Assets written to: MOD/assets


In [17]:
model=load_model("MOD")
#generate random index
index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
music_pattern = x_test[index]

out_pred=[] #it will store predicted notes

#iterate till 200 note is generated
for i in range(200):

  #reshape the music pattern 
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)
  
  #get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))
  #get the note using predicted index and
  #append to the output prediction list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)
  
  #update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]

In [18]:
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    #cast the pattern to Note object apply the offset and 
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_composed_music.mid')

'AI_composed_music.mid'

In [19]:
# playing the file
mf = midi.MidiFile()
mf.open('AI_composed_music.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')